# Writing Custom Context Managers in Python

## Motivation

You might be familiar with the `with` statement, a clean and concise way to handle resources in Python. But have you ever wondered how it works under the hood?  Well, that magic is powered by context managers!

Context managers are a fundamental design pattern in Python that provide a structured approach to resource management.  They ensure that resources are acquired, used properly, and then finally released or cleaned up, even in the presence of errors or exceptions.

This becomes especially important when dealing with resources like files, network connections, or database handles.  By using context managers, we can write cleaner and more reliable code, freeing ourselves from the worry of forgetting to close a file or release a l

In this tutorial, we will go beyond the default provided context managers of Python and learn to write our own. ock.te. te. 

## Understanding context managers in Python

Under the hood, context managers are objects that define two special methods: `__enter__` and `__exit__`.  The `__enter__` method is called when you enter the `with` block, and its return value is assigned to a variable within that block.  The `__exit__` method, on the other hand, is called when the `with` block exits, regardless of whether it finishes normally or with an exception.

This structure ensures proper resource handling. Let's look at some built-in context managers in Python to illustrate this:

1. File Management

Take the classic example of opening a file:

```Python
with open('file.txt', 'r') as file:
  data = file.read()
```

Here, `open('file.txt', 'r')` acts as the context manager.  When you enter the `with` block, the __enter__ method of the file object is called, opening the file and assigning it to the variable file.  You can then use `file.read()` to access the file contents.

Crucially, the `__exit__` method of the file object is guaranteed to be called when the `with` block exits, even if an exception occurs.  This method takes care of closing the file, ensuring you don't leave open file handles lying around.

2. Thread Locking

Moving beyond files, context managers can also be used for thread synchronization using threading.Lock():

```Python
import threading

lock = threading.Lock()
with lock:
  # Critical section
  print("This code is executed under lock protection.")
```

Here, `lock` is a `threading.Lock` object, another context manager. When you enter the `with` block, the `__enter__` method acquires the lock, ensuring only one thread can execute the critical section at a time.  Finally, the `__exit__` method releases the lock upon exiting the with block, allowing other threads to proceed.

3. Database Connections

Similarly, context managers can manage database connections:

```Python
import sqlite3

with sqlite3.connect('database.db') as connection:
  cursor = connection.cursor()
  cursor.execute("SELECT * FROM table")
  rows = cursor.fetchall()
```

The `sqlite3.connect('database.db')` call is a context manager. Entering the `with` block establishes a connection to the database, assigning it to `connection`.  You can then use a `cursor` to interact with the database. The `__exit__` method guarantees the connection is closed when the `with` block exits, preventing resource leaks.

4. Network Sockets

Context managers can even handle network communication:

```python
import socket

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
  s.connect(('localhost', 8080))
  s.sendall(b'Hello, world')
  data = s.recv(1024)
```

Here, `socket.socket(socket.AF_INET, socket.SOCK_STREAM)` creates a socket object that acts as the context manager.  The `__enter__` method creates the socket, and within the `with` block, you can connect, send data, and receive data.  The `__exit__` method ensures the socket is closed properly when done.


5. Directory Scanning

And context managers are versatile! `os.scandir('.')` provides a way to iterate over directory entries:

```python
import os

with os.scandir('.') as entries:
  for entry in entries:
    print(entry.name)
```

`os.scandir('.')` acts as the context manager here.  The `__enter__` method opens a directory scan, and you can iterate over the entries within the with block.  The `__exit__` method cleans up the directory scan upon exiting.

As you can see, the with statement, powered by context managers, simplifies resource management by handling allocation and deallocation automatically.  This keeps your code clean, avoids potential errors, and ensures proper resource usage in your Python applications.

## Writing custom context managers in Python

Let's take this a step further and explore how to craft your own context managers in Python.  This gives you fine-grained control over resource management within your applications.

There are two main approaches to writing custom context managers: class-based and function-based.

### Class-Based Approach
Want more structure and control?  The class-based approach is your friend!  Here, you define a class that implements the special methods `__enter__` and `__exit__`.

Let's look at an example of a `Timer` class that measures execution time:

In [1]:
import time


class Timer:
    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {elapsed_time} seconds")


# Example usage
if __name__ == "__main__":
    with Timer() as timer:
        # Code block to measure the execution time
        time.sleep(2)  # Simulate some time-cning operation

Elapsed time: 2.002065658569336 seconds


Here, the `Timer` class defines the `__enter__` method to capture the start time when you enter the with block.  It returns `self` to allow access to the object within the block.  The `__exit__` method calculates the elapsed time upon exiting the `with` block and prints it.

### Function-Based Approach

Prefer a more concise approach?  The function-based approach might be your style!  Here, you leverage the `contextmanager` decorator from the `contextlib` module.

Let's see how we can achieve the same functionality with a function:

In [3]:
import time
from contextlib import contextmanager


@contextmanager
def timer():
    start_time = time.time()
    yield
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time: {elapsed_time} seconds")


# Example usage
if __name__ == "__main__":
    with timer():
        time.sleep(2)

Elapsed time: 2.002091884613037 seconds


The `@contextmanager` decorator transforms the timer function into a context manager.  Inside the function, `start_time` is captured, and the `yield` statement pauses execution, allowing code within the `with` block to run.  Finally, `__exit__` functionality is achieved by capturing the end time and printing the elapsed time.Essentially, you write the logic for the `__enter__` before the `yield` keyword whereas the logic for `__exit__` comes after. 
Both approaches achieve the same outcome, but the choice depends on your preference for structure and readabilityts!

### Practical examples of context managers

In [ ]:
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None

    def __enter__(self):
        self.file = open(self.filename, self.mode)
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        if self.file:
            self.file.close()


# Example usage
if __name__ == "__main__":
    with FileManager("example.txt", "w") as file:
        file.write("Hello, world!")

In [1]:
import sqlite3


class DatabaseConnection:
    def __init__(self, database_name):
        self.database_name = database_name
        self.connection = None

    def __enter__(self):
        self.connection = sqlite3.connect(self.database_name)
        return self.connection

    def __exit__(self, exc_type, exc_value, traceback):
        if self.connection:
            self.connection.close()


# Example usage
def create_table():
    with DatabaseConnection("example.db") as connection:
        cursor = connection.cursor()
        cursor.execute(
            """CREATE TABLE IF NOT EXISTS users (
                            id INTEGER PRIMARY KEY,
                            username TEXT,
                            email TEXT)"""
        )


def insert_data(username, email):
    with DatabaseConnection("example.db") as connection:
        cursor = connection.cursor()
        cursor.execute(
            "INSERT INTO users (username, email) VALUES (?, ?)", (username, email)
        )
        connection.commit()


def fetch_data():
    with DatabaseConnection("example.db") as connection:
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM users")
        return cursor.fetchall()


if __name__ == "__main__":
    create_table()
    insert_data("john_doe", "john@example.com")
    insert_data("jane_doe", "jane@example.com")
    users = fetch_data()
    print("Users in the database:")
    for user in users:
        print(user)

Users in the database:
(1, 'john_doe', 'john@example.com')
(2, 'jane_doe', 'jane@example.com')


## Advanced topics related to context managers in Python

### Error handling

### Nesting context managers

## Conclusion